In [1]:
import sys
sys.path.append('/content/drive/MyDrive/scripts')

In [17]:
import os
import pandas as pd
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from torchvision import models, transforms



class MultiDayDataset(Dataset):
    def __init__(self, image_paths, irradiance_values, target_size=(224,224)):
        self.image_paths = image_paths
        self.irradiance_values = irradiance_values
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(target_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_tensor = self.transform(img)
        irradiance = torch.tensor(self.irradiance_values[idx], dtype=torch.float32)
        return img_tensor, irradiance

def get_multi_day_dataset(image_dirs, irradiance_files):
    image_paths = []
    irradiance_values = []
    for img_dir, irr_file in zip(image_dirs, irradiance_files):
        if irr_file.endswith('.csv'):
            df = pd.read_csv(irr_file)
            values = df.iloc[:, 1].values.astype(np.float32)
        else:
            values = np.loadtxt(irr_file, delimiter=',')[:, 1].astype(np.float32)

        files = sorted([f for f in os.listdir(img_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])
        files = files[:len(values)]
        image_paths.extend([os.path.join(img_dir, f) for f in files])
        irradiance_values.extend(values[:len(files)])
    return MultiDayDataset(image_paths, irradiance_values)


class EfficientNetRegression(nn.Module):
    def __init__(self, num_classes=1, model_name='efficientnet_b0', pretrained=True):
        super(EfficientNetRegression, self).__init__()
        # Use weights argument instead of pretrained
        if pretrained:
            weights = models.EfficientNet_B0_Weights.IMAGENET1K_V1
        else:
            weights = None
        backbone = getattr(models, model_name)(weights=weights)
        self.features = backbone.features
        self.pooling = nn.AdaptiveAvgPool2d(1)
        in_features = backbone.classifier[1].in_features
        self.regressor = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(in_features, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.pooling(x)
        x = self.regressor(x)
        return x



class CNNTrainer:
    def __init__(self, config=None):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")
        self.config = {
            'learning_rate': 1e-4,
            'batch_size': 64,
            'num_epochs': 20,
            'weight_decay': 1e-4,
            'scheduler_patience': 5,
            'early_stopping_patience': 10,
            'save_dir': 'models',
            'log_dir': 'logs'
        }
        if config:
            self.config.update(config)
        os.makedirs(self.config['save_dir'], exist_ok=True)
        os.makedirs(self.config['log_dir'], exist_ok=True)
        self.model = EfficientNetRegression().to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(),
                                    lr=self.config['learning_rate'],
                                    weight_decay=self.config['weight_decay'])
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(self.optimizer,
                                                             mode='min',
                                                             patience=self.config['scheduler_patience'],
                                                             factor=0.5)
        self.criterion = nn.MSELoss()
        self.train_losses = []
        self.val_losses = []
        self.best_val_loss = float('inf')
        self.patience_counter = 0

    def train_epoch(self, train_loader):
        self.model.train()
        total_loss = 0.0
        with tqdm(train_loader, desc="Training") as pbar:
            for images, targets in pbar:
                images, targets = images.to(self.device), targets.to(self.device)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs.squeeze(), targets)
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()
                pbar.set_postfix({'Loss': f'{loss.item():.4f}'})
        return total_loss / len(train_loader)

    def validate_epoch(self, val_loader):
        self.model.eval()
        total_loss = 0.0
        predictions, targets_all = [], []
        with torch.no_grad():
            with tqdm(val_loader, desc="Validation") as pbar:
                for images, targets in pbar:
                    images, targets = images.to(self.device), targets.to(self.device)
                    outputs = self.model(images)
                    loss = self.criterion(outputs.squeeze(), targets)
                    total_loss += loss.item()
                    predictions.extend(outputs.squeeze().cpu().numpy())
                    targets_all.extend(targets.cpu().numpy())
                    pbar.set_postfix({'Val Loss': f'{loss.item():.4f}'})
        avg_loss = total_loss / len(val_loader)
        predictions = np.array(predictions)
        targets_all = np.array(targets_all)
        rmse = np.sqrt(mean_squared_error(targets_all, predictions))
        mae = mean_absolute_error(targets_all, predictions)
        return avg_loss, rmse, mae

    def train(self, train_loader, val_loader):
        print(f"Starting training for {self.config['num_epochs']} epochs...")
        print(f"Model parameters: {sum(p.numel() for p in self.model.parameters()):,}")
        for epoch in range(self.config['num_epochs']):
            print(f"\nEpoch {epoch+1}/{self.config['num_epochs']}")
            train_loss = self.train_epoch(train_loader)
            self.train_losses.append(train_loss)
            print(f"Train Loss: {train_loss:.4f}")
            val_loss, rmse, mae = self.validate_epoch(val_loader)
            self.val_losses.append(val_loss)
            print(f"Val Loss: {val_loss:.4f}, RMSE: {rmse:.2f}, MAE: {mae:.2f}")
            self.scheduler.step(val_loss)
            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                self.patience_counter = 0
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'val_loss': val_loss,
                    'config': self.config
                }, os.path.join(self.config['save_dir'], 'best_efficientnet_model.pth'))
                print(f"New best model saved!")
            else:
                self.patience_counter += 1
            if self.patience_counter >= self.config['early_stopping_patience']:
                print(f"Early stopping triggered after {epoch+1} epochs")
                break
        plt.figure(figsize=(10,6))
        plt.plot(self.train_losses, label='Train Loss')
        plt.plot(self.val_losses, label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('MSE Loss')
        plt.title('EfficientNet Training Progress')
        plt.legend()
        plt.grid(True)
        plt.show()


# def train_efficientnet_nowcasting():
#     config = {
#         'learning_rate': 1e-4,
#         'batch_size': 64,
#         'num_epochs': 25,
#         'image_dirs': [
#             '/content/drive/MyDrive/data/processed/2019_01_15',
#             '/content/drive/MyDrive/data/processed/2019_01_16',
#             '/content/drive/MyDrive/data/processed/2019_01_17',
#             '/content/drive/MyDrive/data/processed/2019_01_18',
#             '/content/drive/MyDrive/data/processed/2019_01_19',
#             '/content/drive/MyDrive/data/processed/2019_01_20'

#         ],
#         'irradiance_files': [
#             '/content/drive/MyDrive/GIRASOL_DATASET/2019_01_15/pyranometer/2019_01_15.csv',
#             '/content/drive/MyDrive/GIRASOL_DATASET/2019_01_16/pyranometer/2019_01_16.csv',
#             '/content/drive/MyDrive/GIRASOL_DATASET/2019_01_17/pyranometer/2019_01_17.csv',
#             '/content/drive/MyDrive/GIRASOL_DATASET/2019_01_18/pyranometer/2019_01_18.csv',
#             '/content/drive/MyDrive/GIRASOL_DATASET/2019_01_19/pyranometer/2019_01_19.csv',
#             '/content/drive/MyDrive/GIRASOL_DATASET/2019_01_20/pyranometer/2019_01_20.csv',

#         ]
#     }
#     print("Loading multi-day dataset...")
#     dataset = get_multi_day_dataset(config['image_dirs'], config['irradiance_files'])
#     train_size = int(0.8 * len(dataset))
#     val_size = len(dataset) - train_size
#     train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
#     train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=2)
#     val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=2)
#     print(f"Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}")
#     trainer = CNNTrainer(config=config)
#     trainer.train(train_loader, val_loader)
#     print("EfficientNet nowcasting training completed!")

# # ============================
# # 6. Run Training
# # ============================
# train_efficientnet_nowcasting()

In [42]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from torch.utils.data import DataLoader, Dataset
import json
from datetime import datetime
import torch.nn as nn
import torch.serialization

from cnn_model import SolarCNNRegression, SolarCNNWithFeatureExtraction
from lstm_model import HybridCNNLSTM
from solar_datasets import SolarIrradianceDataset, SolarTimeSeriesDataset, SolarSequenceDataset


# ---------------------------
# Dataset
# ---------------------------
class MultiDayTimeSeriesDataset(Dataset):
    def __init__(self, irradiance_file, sequence_length=20, forecast_horizon=4, value_col_index=1):
        df = pd.read_csv(irradiance_file)
        vals = df.iloc[:, value_col_index].astype(np.float32).values
        self.series = vals
        self.seq_len = sequence_length
        self.horizon = forecast_horizon
        self.length = len(self.series) - self.seq_len - self.horizon + 1
        if self.length < 1:
            raise ValueError("Not enough data samples for given sequence_length + forecast_horizon.")
    def __len__(self):
        return self.length
    def __getitem__(self, idx):
        seq = self.series[idx : idx + self.seq_len]
        tgt = self.series[idx + self.seq_len : idx + self.seq_len + self.horizon]
        seq = torch.tensor(seq, dtype=torch.float32).unsqueeze(-1)
        tgt = torch.tensor(tgt, dtype=torch.float32)
        return seq, tgt


# ---------------------------
# LSTM Model
# ---------------------------
class SolarLSTMForecasting(nn.Module):
    def __init__(self, input_size=1, hidden_size=128, num_layers=2, output_size=4, dropout=0.2):
        super(SolarLSTMForecasting, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True,
                            dropout=dropout if num_layers > 1 else 0.0)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        out, _ = self.lstm(x)
        last = out[:, -1, :]
        return self.fc(last)


# ---------------------------
# Safe checkpoint loader
# ---------------------------
def safe_load_checkpoint(model, model_path, device):
    import torch.serialization
    import numpy as np
    try:
        torch.serialization.add_safe_globals([np.core.multiarray.scalar, np.dtype, np.float64])
    except Exception:
        pass

    ckpt = torch.load(model_path, map_location=device, weights_only=False)
    if isinstance(ckpt, dict) and 'model_state_dict' in ckpt:
        state_dict = ckpt['model_state_dict']
    elif isinstance(ckpt, dict):
        state_dict = ckpt
    else:
        raise RuntimeError("Unexpected checkpoint format.")

    model_dict = model.state_dict()
    filtered = {}
    skipped = []
    for k, v in state_dict.items():
        if k in model_dict:
            if isinstance(v, torch.Tensor) and v.shape == model_dict[k].shape:
                filtered[k] = v
            else:
                skipped.append((k, getattr(v, 'shape', None), model_dict[k].shape))
    model_dict.update(filtered)
    model.load_state_dict(model_dict)
    if skipped:
        print("Warning: some keys were skipped due to shape mismatch:")
        for item in skipped[:10]:
            print("  ", item)
    return ckpt


# ---------------------------
# Evaluation class
# ---------------------------
class ModelEvaluator:
    def __init__(self, device=None):
        self.device = device or torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")
        self.results = {}

    def load_model(self, model_class, checkpoint_path, **kwargs):
        model = model_class(**kwargs).to(self.device)
        checkpoint = torch.load(checkpoint_path, map_location=self.device, weights_only=False)
        model.load_state_dict(checkpoint['model_state_dict'])
        model.eval()
        return model, checkpoint

    def evaluate_cnn(self, model_path, test_dataset):
        print("\n=== Evaluating CNN Nowcasting Model ===")
        model, checkpoint = self.load_model(SolarCNNRegression, model_path)
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2 )
        predictions, targets = [], []
        with torch.no_grad():
            for images, batch_targets in test_loader:
                images, batch_targets = images.to(self.device), batch_targets.to(self.device)
                outputs = model(images)
                predictions.extend(outputs.squeeze().cpu().numpy())
                targets.extend(batch_targets.cpu().numpy())

        predictions = np.array(predictions)
        targets = np.array(targets)

        mse = mean_squared_error(targets, predictions)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(targets, predictions)

        self.results['cnn'] = {
            'rmse': rmse,
            'mae': mae,
            'predictions': predictions.tolist(),
            'targets': targets.tolist(),
            'num_samples': len(predictions)
        }

        print(f"CNN Results:")
        print(f"  RMSE: {rmse:.2f} W/m²")
        print(f"  MAE: {mae:.2f} W/m²")
        print(f"  Samples: {len(predictions)}")

        return rmse, mae, predictions, targets

    def evaluate_lstm(self, model_path, test_dataset):
        print("\n=== Evaluating LSTM Forecasting Model ===")
        checkpoint = torch.load(model_path, map_location=self.device, weights_only=False)
        config = checkpoint.get('config', {})
        model = SolarLSTMForecasting(
            input_size=1,
            hidden_size=config.get('lstm_hidden_size', 128),
            num_layers=config.get('lstm_num_layers', 2),
            output_size=config.get('forecast_horizon', 4),
            dropout=config.get('dropout', 0.2)
        ).to(self.device)

        safe_load_checkpoint(model, model_path, self.device)
        model.eval()
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)
        all_predictions, all_targets = [], []
        with torch.no_grad():
            for sequences, targets in test_loader:
                sequences, targets = sequences.to(self.device), targets.to(self.device)
                outputs = model(sequences)
                all_predictions.append(outputs.cpu().numpy())
                all_targets.append(targets.cpu().numpy())

        predictions = np.concatenate(all_predictions, axis=0)
        targets = np.concatenate(all_targets, axis=0)

        overall_rmse = np.sqrt(mean_squared_error(targets.flatten(), predictions.flatten()))
        overall_mae = mean_absolute_error(targets.flatten(), predictions.flatten())

        forecast_step_metrics = []
        if predictions.shape[1] > 0:
            forecast_horizon = predictions.shape[1]
            for step in range(forecast_horizon):
                step_rmse = np.sqrt(mean_squared_error(targets[:, step], predictions[:, step]))
                step_mae = mean_absolute_error(targets[:, step], predictions[:, step])
                forecast_step_metrics.append({
                    'step': step + 1,
                    'rmse': step_rmse,
                    'mae': step_mae
                })

        print("\n=== Overall Metrics ===")
        print(f"RMSE: {overall_rmse:.4f}")
        print(f"MAE : {overall_mae:.4f}")

        self.results['lstm'] = {
            'overall_rmse': overall_rmse,
            'overall_mae': overall_mae,
            'predictions': predictions.tolist(),
            'targets': targets.tolist(),
            'num_samples': len(predictions),
            'step_metrics': forecast_step_metrics # Add step metrics
        }

        return overall_rmse, overall_mae, predictions, targets

    def evaluate_hybrid(self, model_path, test_dataset):
        print("\n=== Evaluating Hybrid CNN-LSTM Model ===")
        checkpoint = torch.load(model_path, map_location=self.device, weights_only=False)
        config = checkpoint.get('config', {})
        model, _ = self.load_model(
            HybridCNNLSTM,
            model_path,
            sequence_length=config.get('sequence_length', 20),
            lstm_hidden_size=config.get('lstm_hidden_size', 128),
            forecast_horizon=config.get('forecast_horizon', 4)
        )

        test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=2)
        all_nowcast_pred, all_nowcast_target, all_forecast_pred, all_forecast_target = [], [], [], []
        with torch.no_grad():
            for image_sequences, historical_irradiance, future_irradiance in test_loader:
                image_sequences = image_sequences.to(self.device)
                historical_irradiance = historical_irradiance.to(self.device)
                future_irradiance = future_irradiance.to(self.device)
                nowcasts, forecasts = model(image_sequences)
                all_nowcast_pred.append(nowcasts.squeeze(-1).cpu().numpy())
                all_nowcast_target.append(historical_irradiance.squeeze(-1).cpu().numpy())
                all_forecast_pred.append(forecasts.cpu().numpy())
                all_forecast_target.append(future_irradiance.cpu().numpy())

        nowcast_pred = np.concatenate(all_nowcast_pred, axis=0)
        nowcast_target = np.concatenate(all_nowcast_target, axis=0)
        forecast_pred = np.concatenate(all_forecast_pred, axis=0)
        forecast_target = np.concatenate(all_forecast_target, axis=0)

        nowcast_rmse = np.sqrt(mean_squared_error(nowcast_target.flatten(), nowcast_pred.flatten()))
        nowcast_mae = mean_absolute_error(nowcast_target.flatten(), nowcast_pred.flatten())
        forecast_rmse = np.sqrt(mean_squared_error(forecast_target.flatten(), forecast_pred.flatten()))
        forecast_mae = mean_absolute_error(forecast_target.flatten(), forecast_pred.flatten())

        forecast_step_metrics = []
        forecast_horizon = forecast_pred.shape[1]
        for step in range(forecast_horizon):
            step_rmse = np.sqrt(mean_squared_error(forecast_target[:, step], forecast_pred[:, step]))
            step_mae = mean_absolute_error(forecast_target[:, step], forecast_pred[:, step])
            forecast_step_metrics.append({
                'step': step + 1,
                'rmse': step_rmse,
                'mae': step_mae
            })

        self.results['hybrid'] = {
            'nowcast_rmse': nowcast_rmse,
            'nowcast_mae': nowcast_mae,
            'forecast_rmse': forecast_rmse,
            'forecast_mae': forecast_mae,
            'forecast_step_metrics': forecast_step_metrics,
            'num_samples': len(forecast_pred)
        }

        print(f"Hybrid Results:")
        print(f"  Nowcast RMSE: {nowcast_rmse:.2f} W/m²")
        print(f"  Nowcast MAE: {nowcast_mae:.2f} W/m²")
        print(f"  Forecast RMSE: {forecast_rmse:.2f} W/m²")
        print(f"  Forecast MAE: {forecast_mae:.2f} W/m²")
        print(f"  Samples: {len(forecast_pred)}")

        return (nowcast_rmse, nowcast_mae, forecast_rmse, forecast_mae,
                forecast_step_metrics, nowcast_pred, forecast_pred)


    def plot_results(self, save_dir='evaluation_plots'):
        """Create visualization plots"""
        os.makedirs(save_dir, exist_ok=True)

        # CNN scatter plot
        if 'cnn' in self.results:
            plt.figure(figsize=(10, 8))

            targets = np.array(self.results['cnn']['targets'])
            predictions = np.array(self.results['cnn']['predictions'])

            plt.subplot(2, 2, 1)
            plt.scatter(targets, predictions, alpha=0.6, s=20)
            plt.plot([targets.min(), targets.max()], [targets.min(), targets.max()], 'r--', lw=2)
            plt.xlabel('Actual Irradiance (W/m²)')
            plt.ylabel('Predicted Irradiance (W/m²)')
            plt.title(f'CNN Nowcasting (RMSE: {self.results["cnn"]["rmse"]:.2f} W/m²)')
            plt.grid(True, alpha=0.3)

            # Residuals plot
            plt.subplot(2, 2, 2)
            residuals = predictions - targets
            plt.scatter(targets, residuals, alpha=0.6, s=20)
            plt.axhline(y=0, color='r', linestyle='--')
            plt.xlabel('Actual Irradiance (W/m²)')
            plt.ylabel('Residuals (W/m²)')
            plt.title('CNN Residuals')
            plt.grid(True, alpha=0.3)

            plt.tight_layout()
            plt.savefig(f'{save_dir}/cnn_evaluation.png', dpi=300, bbox_inches='tight')
            plt.show()
            plt.close()

        # LSTM overall performance
        if 'lstm' in self.results:
            overall_rmse = self.results['lstm']['overall_rmse']
            overall_mae = self.results['lstm']['overall_mae']

            plt.figure(figsize=(8, 6))
            bar_labels = ['RMSE', 'MAE']
            error_values = [overall_rmse, overall_mae]

            plt.bar(bar_labels, error_values, color=['skyblue', 'lightcoral'], alpha=0.7)
            plt.ylabel('Error (W/m²)')
            plt.title('LSTM Overall Performance')
            plt.grid(True, alpha=0.3, axis='y')

            # Add value labels on bars
            for i, val in enumerate(error_values):
                plt.text(i, val + 1, f'{val:.2f}', ha='center', va='bottom')

            plt.tight_layout()
            plt.savefig(f'{save_dir}/lstm_overall_evaluation.png', dpi=300, bbox_inches='tight')
            plt.show()
            plt.close()


        # Model comparison
        if len(self.results) > 1:
            plt.figure(figsize=(12, 8))

            models = []
            rmse_values = []
            mae_values = []

            if 'cnn' in self.results:
                models.append('CNN\nNowcasting')
                rmse_values.append(self.results['cnn']['rmse'])
                mae_values.append(self.results['cnn']['mae'])

            if 'lstm' in self.results:
                models.append('LSTM\nForecasting')
                rmse_values.append(self.results['lstm']['overall_rmse'])
                mae_values.append(self.results['lstm']['overall_mae'])

            if 'hybrid' in self.results:
                models.append('Hybrid\nNowcast')
                rmse_values.append(self.results['hybrid']['nowcast_rmse'])
                mae_values.append(self.results['hybrid']['nowcast_mae'])

                models.append('Hybrid\nForecast')
                rmse_values.append(self.results['hybrid']['forecast_rmse'])
                mae_values.append(self.results['hybrid']['forecast_mae'])

            x = np.arange(len(models))
            width = 0.35

            plt.subplot(1, 1, 1)
            plt.bar(x - width/2, rmse_values, width, label='RMSE', color='skyblue', alpha=0.8)
            plt.bar(x + width/2, mae_values, width, label='MAE', color='lightcoral', alpha=0.8)

            plt.xlabel('Models')
            plt.ylabel('Error (W/m²)')
            plt.title('Model Performance Comparison')
            plt.xticks(x, models)
            plt.legend()
            plt.grid(True, alpha=0.3)

            # Add value labels on bars
            for i, (rmse, mae) in enumerate(zip(rmse_values, mae_values)):
                plt.text(i - width/2, rmse + 1, f'{rmse:.1f}', ha='center', va='bottom')
                plt.text(i + width/2, mae + 1, f'{mae:.1f}', ha='center', va='bottom')

            plt.tight_layout()
            plt.savefig(f'{save_dir}/model_comparison.png', dpi=300, bbox_inches='tight')
            plt.show()
            plt.close()

        print(f"\nPlots saved to {save_dir}/")

    def save_results(self, filename='evaluation_results.json'):
        results_with_metadata = {
            'evaluation_timestamp': datetime.now().isoformat(),
            'device': str(self.device),
            'results': self.results
        }
        with open(filename, 'w') as f:
            json.dump(results_with_metadata, f, indent=2)
        print(f"\nResults saved to {filename}")

    def generate_report(self, filename='evaluation_report.txt'):
        with open(filename, 'w') as f:
            f.write("="*80 + "\n")
            f.write("SOLAR IRRADIANCE FORECASTING - MODEL EVALUATION REPORT\n")
            f.write("="*80 + "\n")
            f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Device: {self.device}\n\n")
            if 'cnn' in self.results:
                r = self.results['cnn']
                f.write("CNN NOWCASTING MODEL\n")
                f.write("-"*30 + "\n")
                f.write(f"RMSE: {r['rmse']:.2f} W/m²\n")
                f.write(f"MAE: {r['mae']:.2f} W/m²\n\n")
            if 'lstm' in self.results:
                r = self.results['lstm']
                f.write("LSTM FORECASTING MODEL\n")
                f.write("-"*30 + "\n")
                f.write(f"RMSE: {r['overall_rmse']:.2f} W/m²\n")
                f.write(f"MAE: {r['overall_mae']:.2f} W/m²\n\n")
            if 'hybrid' in self.results:
                r = self.results['hybrid']
                f.write("HYBRID CNN-LSTM MODEL\n")
                f.write("-"*30 + "\n")
                f.write(f"Nowcast RMSE: {r['nowcast_rmse']:.2f} W/m²\n")
                f.write(f"Nowcast MAE: {r['nowcast_mae']:.2f} W/m²\n")
                f.write(f"Forecast RMSE: {r['forecast_rmse']:.2f} W/m²\n")
                f.write(f"Forecast MAE: {r['forecast_mae']:.2f} W/m²\n\n")
        print(f"Report saved to {filename}")




In [13]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from torch.utils.data import DataLoader, Dataset
import json
from datetime import datetime
import torch.nn as nn
import torch.serialization

from cnn_model import SolarCNNRegression, SolarCNNWithFeatureExtraction
from lstm_model import HybridCNNLSTM
from solar_datasets import SolarIrradianceDataset, SolarTimeSeriesDataset, SolarSequenceDataset


# ---------------------------
# Dataset
# ---------------------------
class MultiDayTimeSeriesDataset(Dataset):
    def __init__(self, irradiance_file, sequence_length=20, forecast_horizon=4, value_col_index=1):
        df = pd.read_csv(irradiance_file)
        vals = df.iloc[:, value_col_index].astype(np.float32).values
        self.series = vals
        self.seq_len = sequence_length
        self.horizon = forecast_horizon
        self.length = len(self.series) - self.seq_len - self.horizon + 1
        if self.length < 1:
            raise ValueError("Not enough data samples for given sequence_length + forecast_horizon.")
    def __len__(self):
        return self.length
    def __getitem__(self, idx):
        seq = self.series[idx : idx + self.seq_len]
        tgt = self.series[idx + self.seq_len : idx + self.seq_len + self.horizon]
        seq = torch.tensor(seq, dtype=torch.float32).unsqueeze(-1)
        tgt = torch.tensor(tgt, dtype=torch.float32)
        return seq, tgt


# ---------------------------
# LSTM Model
# ---------------------------
class SolarLSTMForecasting(nn.Module):
    def __init__(self, input_size=1, hidden_size=128, num_layers=2, output_size=4, dropout=0.2):
        super(SolarLSTMForecasting, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True,
                            dropout=dropout if num_layers > 1 else 0.0)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        out, _ = self.lstm(x)
        last = out[:, -1, :]
        return self.fc(last)


# ---------------------------
# Safe checkpoint loader
# ---------------------------
def safe_load_checkpoint(model, model_path, device):
    import torch.serialization
    import numpy as np
    try:
        torch.serialization.add_safe_globals([np.core.multiarray.scalar, np.dtype, np.float64])
    except Exception:
        pass

    ckpt = torch.load(model_path, map_location=device, weights_only=False)
    if isinstance(ckpt, dict) and 'model_state_dict' in ckpt:
        state_dict = ckpt['model_state_dict']
    elif isinstance(ckpt, dict):
        state_dict = ckpt
    else:
        raise RuntimeError("Unexpected checkpoint format.")

    model_dict = model.state_dict()
    filtered = {}
    skipped = []
    for k, v in state_dict.items():
        if k in model_dict:
            if isinstance(v, torch.Tensor) and v.shape == model_dict[k].shape:
                filtered[k] = v
            else:
                skipped.append((k, getattr(v, 'shape', None), model_dict[k].shape))
    model_dict.update(filtered)
    model.load_state_dict(model_dict)
    if skipped:
        print("Warning: some keys were skipped due to shape mismatch:")
        for item in skipped[:10]:
            print("  ", item)
    return ckpt


# ---------------------------
# Evaluation class
# ---------------------------
class ModelEvaluator:
    def __init__(self, device=None):
        self.device = device or torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")
        self.results = {}

    def load_model(self, model_class, checkpoint_path, **kwargs):
        model = model_class(**kwargs).to(self.device)
        checkpoint = torch.load(checkpoint_path, map_location=self.device, weights_only=False)
        model.load_state_dict(checkpoint['model_state_dict'])
        model.eval()
        return model, checkpoint

    def evaluate_cnn(self, model_path, test_dataset):
        print("\n=== Evaluating CNN Nowcasting Model ===")
        model, checkpoint = self.load_model(EfficientNetRegression, model_path)
        test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)
        predictions, targets = [], []
        with torch.no_grad():
            for images, batch_targets in test_loader:
                images, batch_targets = images.to(self.device), batch_targets.to(self.device)
                outputs = model(images)
                predictions.extend(outputs.squeeze().cpu().numpy())
                targets.extend(batch_targets.cpu().numpy())

        predictions = np.array(predictions)
        targets = np.array(targets)

        mse = mean_squared_error(targets, predictions)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(targets, predictions)

        self.results['cnn'] = {
            'rmse': rmse,
            'mae': mae,
            'predictions': predictions.tolist(),
            'targets': targets.tolist(),
            'num_samples': len(predictions)
        }

        print(f"CNN Results:")
        print(f"  RMSE: {rmse:.2f} W/m²")
        print(f"  MAE: {mae:.2f} W/m²")
        print(f"  Samples: {len(predictions)}")

        return rmse, mae, predictions, targets

    def evaluate_model(self, model, model_name, test_dataset, batch_size=16):
        """Evaluate a single model on test data"""
        print(f"\n=== Evaluating {model_name} Model ===")
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        preds, targets = [], []

        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(self.device), labels.to(self.device)
                outputs = model(images).squeeze()
                preds.extend(outputs.cpu().numpy())
                targets.extend(labels.cpu().numpy())

        preds, targets = np.array(preds), np.array(targets)
        mse = mean_squared_error(targets, preds)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(targets, preds)

        self.results[model_name] = {
            "RMSE": rmse,
            "MAE": mae,
            "Samples": len(preds),
            "Predictions": preds.tolist(),
            "Targets": targets.tolist(),
        }

        print(f"RMSE: {rmse:.2f} | MAE: {mae:.2f} | Samples: {len(preds)}")
        print("------------------------------------------------------------")
        return preds, targets

    def evaluate_lstm(self, model_path, test_dataset):
        print("\n=== Evaluating LSTM Forecasting Model ===")
        checkpoint = torch.load(model_path, map_location=self.device, weights_only=False)
        config = checkpoint.get('config', {})
        model = SolarLSTMForecasting(
            input_size=1,
            hidden_size=config.get('lstm_hidden_size', 128),
            num_layers=config.get('lstm_num_layers', 2),
            output_size=config.get('forecast_horizon', 4),
            dropout=config.get('dropout', 0.2)
        ).to(self.device)

        safe_load_checkpoint(model, model_path, self.device)
        model.eval()
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)
        all_predictions, all_targets = [], []
        with torch.no_grad():
            for sequences, targets in test_loader:
                sequences, targets = sequences.to(self.device), targets.to(self.device)
                outputs = model(sequences)
                all_predictions.append(outputs.cpu().numpy())
                all_targets.append(targets.cpu().numpy())

        predictions = np.concatenate(all_predictions, axis=0)
        targets = np.concatenate(all_targets, axis=0)

        overall_rmse = np.sqrt(mean_squared_error(targets.flatten(), predictions.flatten()))
        overall_mae = mean_absolute_error(targets.flatten(), predictions.flatten())

        forecast_step_metrics = []
        if predictions.shape[1] > 0:
            forecast_horizon = predictions.shape[1]
            for step in range(forecast_horizon):
                step_rmse = np.sqrt(mean_squared_error(targets[:, step], predictions[:, step]))
                step_mae = mean_absolute_error(targets[:, step], predictions[:, step])
                forecast_step_metrics.append({
                    'step': step + 1,
                    'rmse': step_rmse,
                    'mae': step_mae
                })

        print("\n=== Overall Metrics ===")
        print(f"RMSE: {overall_rmse:.4f}")
        print(f"MAE : {overall_mae:.4f}")

        self.results['lstm'] = {
            'overall_rmse': overall_rmse,
            'overall_mae': overall_mae,
            'predictions': predictions.tolist(),
            'targets': targets.tolist(),
            'num_samples': len(predictions),
            'step_metrics': forecast_step_metrics # Add step metrics
        }

        return overall_rmse, overall_mae, predictions, targets

    def evaluate_hybrid(self, model_path, test_dataset):
        print("\n=== Evaluating Hybrid CNN-LSTM Model ===")
        checkpoint = torch.load(model_path, map_location=self.device, weights_only=False)
        config = checkpoint.get('config', {})
        model, _ = self.load_model(
            HybridCNNLSTM,
            model_path,
            sequence_length=config.get('sequence_length', 20),
            lstm_hidden_size=config.get('lstm_hidden_size', 128),
            forecast_horizon=config.get('forecast_horizon', 4)
        )

        test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=2)
        all_nowcast_pred, all_nowcast_target, all_forecast_pred, all_forecast_target = [], [], [], []
        with torch.no_grad():
            for image_sequences, historical_irradiance, future_irradiance in test_loader:
                image_sequences = image_sequences.to(self.device)
                historical_irradiance = historical_irradiance.to(self.device)
                future_irradiance = future_irradiance.to(self.device)
                nowcasts, forecasts = model(image_sequences)
                all_nowcast_pred.append(nowcasts.squeeze(-1).cpu().numpy())
                all_nowcast_target.append(historical_irradiance.squeeze(-1).cpu().numpy())
                all_forecast_pred.append(forecasts.cpu().numpy())
                all_forecast_target.append(future_irradiance.cpu().numpy())

        nowcast_pred = np.concatenate(all_nowcast_pred, axis=0)
        nowcast_target = np.concatenate(all_nowcast_target, axis=0)
        forecast_pred = np.concatenate(all_forecast_pred, axis=0)
        forecast_target = np.concatenate(all_forecast_target, axis=0)

        nowcast_rmse = np.sqrt(mean_squared_error(nowcast_target.flatten(), nowcast_pred.flatten()))
        nowcast_mae = mean_absolute_error(nowcast_target.flatten(), nowcast_pred.flatten())
        forecast_rmse = np.sqrt(mean_squared_error(forecast_target.flatten(), forecast_pred.flatten()))
        forecast_mae = mean_absolute_error(forecast_target.flatten(), forecast_pred.flatten())

        forecast_step_metrics = []
        forecast_horizon = forecast_pred.shape[1]
        for step in range(forecast_horizon):
            step_rmse = np.sqrt(mean_squared_error(forecast_target[:, step], forecast_pred[:, step]))
            step_mae = mean_absolute_error(forecast_target[:, step], forecast_pred[:, step])
            forecast_step_metrics.append({
                'step': step + 1,
                'rmse': step_rmse,
                'mae': step_mae
            })

        self.results['hybrid'] = {
            'nowcast_rmse': nowcast_rmse,
            'nowcast_mae': nowcast_mae,
            'forecast_rmse': forecast_rmse,
            'forecast_mae': forecast_mae,
            'forecast_step_metrics': forecast_step_metrics,
            'num_samples': len(forecast_pred)
        }

        print(f"Hybrid Results:")
        print(f"  Nowcast RMSE: {nowcast_rmse:.2f} W/m²")
        print(f"  Nowcast MAE: {nowcast_mae:.2f} W/m²")
        print(f"  Forecast RMSE: {forecast_rmse:.2f} W/m²")
        print(f"  Forecast MAE: {forecast_mae:.2f} W/m²")
        print(f"  Samples: {len(forecast_pred)}")

        return (nowcast_rmse, nowcast_mae, forecast_rmse, forecast_mae,
                forecast_step_metrics, nowcast_pred, forecast_pred)


    def plot_results(self, save_dir='evaluation_plots'):
        """Create visualization plots"""
        os.makedirs(save_dir, exist_ok=True)

        # CNN scatter plot
        if 'cnn' in self.results:
            plt.figure(figsize=(10, 8))

            targets = np.array(self.results['cnn']['targets'])
            predictions = np.array(self.results['cnn']['predictions'])

            plt.subplot(2, 2, 1)
            plt.scatter(targets, predictions, alpha=0.6, s=20)
            plt.plot([targets.min(), targets.max()], [targets.min(), targets.max()], 'r--', lw=2)
            plt.xlabel('Actual Irradiance (W/m²)')
            plt.ylabel('Predicted Irradiance (W/m²)')
            plt.title(f'CNN Nowcasting (RMSE: {self.results["cnn"]["rmse"]:.2f} W/m²)')
            plt.grid(True, alpha=0.3)

            # Residuals plot
            plt.subplot(2, 2, 2)
            residuals = predictions - targets
            plt.scatter(targets, residuals, alpha=0.6, s=20)
            plt.axhline(y=0, color='r', linestyle='--')
            plt.xlabel('Actual Irradiance (W/m²)')
            plt.ylabel('Residuals (W/m²)')
            plt.title('CNN Residuals')
            plt.grid(True, alpha=0.3)

            plt.tight_layout()
            plt.savefig(f'{save_dir}/cnn_evaluation.png', dpi=300, bbox_inches='tight')
            plt.close()

        # LSTM overall performance
        if 'lstm' in self.results:
            overall_rmse = self.results['lstm']['overall_rmse']
            overall_mae = self.results['lstm']['overall_mae']

            plt.figure(figsize=(8, 6))
            bar_labels = ['RMSE', 'MAE']
            error_values = [overall_rmse, overall_mae]

            plt.bar(bar_labels, error_values, color=['skyblue', 'lightcoral'], alpha=0.7)
            plt.ylabel('Error (W/m²)')
            plt.title('LSTM Overall Performance')
            plt.grid(True, alpha=0.3, axis='y')

            # Add value labels on bars
            for i, val in enumerate(error_values):
                plt.text(i, val + 1, f'{val:.2f}', ha='center', va='bottom')

            plt.tight_layout()
            plt.savefig(f'{save_dir}/lstm_overall_evaluation.png', dpi=300, bbox_inches='tight')
            plt.close()


        # Model comparison
        if len(self.results) > 1:
            plt.figure(figsize=(12, 8))

            models = []
            rmse_values = []
            mae_values = []

            if 'cnn' in self.results:
                models.append('CNN\nNowcasting')
                rmse_values.append(self.results['cnn']['rmse'])
                mae_values.append(self.results['cnn']['mae'])

            if 'lstm' in self.results:
                models.append('LSTM\nForecasting')
                rmse_values.append(self.results['lstm']['overall_rmse'])
                mae_values.append(self.results['lstm']['overall_mae'])

            if 'hybrid' in self.results:
                models.append('Hybrid\nNowcast')
                rmse_values.append(self.results['hybrid']['nowcast_rmse'])
                mae_values.append(self.results['hybrid']['nowcast_mae'])

                models.append('Hybrid\nForecast')
                rmse_values.append(self.results['hybrid']['forecast_rmse'])
                mae_values.append(self.results['hybrid']['forecast_mae'])

            x = np.arange(len(models))
            width = 0.35

            plt.subplot(1, 1, 1)
            plt.bar(x - width/2, rmse_values, width, label='RMSE', color='skyblue', alpha=0.8)
            plt.bar(x + width/2, mae_values, width, label='MAE', color='lightcoral', alpha=0.8)

            plt.xlabel('Models')
            plt.ylabel('Error (W/m²)')
            plt.title('Model Performance Comparison')
            plt.xticks(x, models)
            plt.legend()
            plt.grid(True, alpha=0.3)

            # Add value labels on bars
            for i, (rmse, mae) in enumerate(zip(rmse_values, mae_values)):
                plt.text(i - width/2, rmse + 1, f'{rmse:.1f}', ha='center', va='bottom')
                plt.text(i + width/2, mae + 1, f'{mae:.1f}', ha='center', va='bottom')

            plt.tight_layout()
            plt.savefig(f'{save_dir}/model_comparison.png', dpi=300, bbox_inches='tight')
            plt.close()

        print(f"\nPlots saved to {save_dir}/")

    def save_results(self, filename='evaluation_results.json'):
        results_with_metadata = {
            'evaluation_timestamp': datetime.now().isoformat(),
            'device': str(self.device),
            'results': self.results
        }
        with open(filename, 'w') as f:
            json.dump(results_with_metadata, f, indent=2)
        print(f"\nResults saved to {filename}")

    def generate_report(self, filename='evaluation_report.txt'):
        with open(filename, 'w') as f:
            f.write("="*80 + "\n")
            f.write("SOLAR IRRADIANCE FORECASTING - MODEL EVALUATION REPORT\n")
            f.write("="*80 + "\n")
            f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Device: {self.device}\n\n")
            if 'cnn' in self.results:
                r = self.results['cnn']
                f.write("CNN NOWCASTING MODEL\n")
                f.write("-"*30 + "\n")
                f.write(f"RMSE: {r['rmse']:.2f} W/m²\n")
                f.write(f"MAE: {r['mae']:.2f} W/m²\n\n")
            if 'lstm' in self.results:
                r = self.results['lstm']
                f.write("LSTM FORECASTING MODEL\n")
                f.write("-"*30 + "\n")
                f.write(f"RMSE: {r['overall_rmse']:.2f} W/m²\n")
                f.write(f"MAE: {r['overall_mae']:.2f} W/m²\n\n")
            if 'hybrid' in self.results:
                r = self.results['hybrid']
                f.write("HYBRID CNN-LSTM MODEL\n")
                f.write("-"*30 + "\n")
                f.write(f"Nowcast RMSE: {r['nowcast_rmse']:.2f} W/m²\n")
                f.write(f"Nowcast MAE: {r['nowcast_mae']:.2f} W/m²\n")
                f.write(f"Forecast RMSE: {r['forecast_rmse']:.2f} W/m²\n")
                f.write(f"Forecast MAE: {r['forecast_mae']:.2f} W/m²\n\n")
        print(f"Report saved to {filename}")




In [15]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from torch.utils.data import DataLoader, Dataset
import json
from datetime import datetime
import torch.nn as nn
import torch.serialization

from cnn_model import SolarCNNRegression, SolarCNNWithFeatureExtraction
from lstm_model import HybridCNNLSTM
from solar_datasets import SolarIrradianceDataset, SolarTimeSeriesDataset, SolarSequenceDataset


# ---------------------------
# Dataset
# ---------------------------
class MultiDayTimeSeriesDataset(Dataset):
    def __init__(self, irradiance_file, sequence_length=20, forecast_horizon=4, value_col_index=1):
        df = pd.read_csv(irradiance_file)
        vals = df.iloc[:, value_col_index].astype(np.float32).values
        self.series = vals
        self.seq_len = sequence_length
        self.horizon = forecast_horizon
        self.length = len(self.series) - self.seq_len - self.horizon + 1
        if self.length < 1:
            raise ValueError("Not enough data samples for given sequence_length + forecast_horizon.")
    def __len__(self):
        return self.length
    def __getitem__(self, idx):
        seq = self.series[idx : idx + self.seq_len]
        tgt = self.series[idx + self.seq_len : idx + self.seq_len + self.horizon]
        seq = torch.tensor(seq, dtype=torch.float32).unsqueeze(-1)
        tgt = torch.tensor(tgt, dtype=torch.float32)
        return seq, tgt


# ---------------------------
# LSTM Model
# ---------------------------
class SolarLSTMForecasting(nn.Module):
    def __init__(self, input_size=1, hidden_size=128, num_layers=2, output_size=4, dropout=0.2):
        super(SolarLSTMForecasting, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True,
                            dropout=dropout if num_layers > 1 else 0.0)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        out, _ = self.lstm(x)
        last = out[:, -1, :]
        return self.fc(last)


# ---------------------------
# Safe checkpoint loader
# ---------------------------
def safe_load_checkpoint(model, model_path, device):
    import torch.serialization
    import numpy as np
    try:
        torch.serialization.add_safe_globals([np.core.multiarray.scalar, np.dtype, np.float64])
    except Exception:
        pass

    ckpt = torch.load(model_path, map_location=device, weights_only=False)
    if isinstance(ckpt, dict) and 'model_state_dict' in ckpt:
        state_dict = ckpt['model_state_dict']
    elif isinstance(ckpt, dict):
        state_dict = ckpt
    else:
        raise RuntimeError("Unexpected checkpoint format.")

    model_dict = model.state_dict()
    filtered = {}
    skipped = []
    for k, v in state_dict.items():
        if k in model_dict:
            if isinstance(v, torch.Tensor) and v.shape == model_dict[k].shape:
                filtered[k] = v
            else:
                skipped.append((k, getattr(v, 'shape', None), model_dict[k].shape))
    model_dict.update(filtered)
    model.load_state_dict(model_dict)
    if skipped:
        print("Warning: some keys were skipped due to shape mismatch:")
        for item in skipped[:10]:
            print("  ", item)
    return ckpt


# ---------------------------
# Evaluation class
# ---------------------------
class ModelEvaluator:
    def __init__(self, device=None):
        self.device = device or torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")
        self.results = {}

    def load_model(self, model_class, checkpoint_path, **kwargs):
        model = model_class(**kwargs).to(self.device)
        checkpoint = torch.load(checkpoint_path, map_location=self.device, weights_only=False)
        model.load_state_dict(checkpoint['model_state_dict'])
        model.eval()
        return model, checkpoint

    def evaluate_cnn(self, model_path, test_dataset):
        print("\n=== Evaluating CNN Nowcasting Model ===")
        model, checkpoint = self.load_model(SolarCNNRegression, model_path)
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)
        predictions, targets = [], []
        with torch.no_grad():
            for images, batch_targets in test_loader:
                images, batch_targets = images.to(self.device), batch_targets.to(self.device)
                outputs = model(images)
                predictions.extend(outputs.squeeze().cpu().numpy())
                targets.extend(batch_targets.cpu().numpy())

        predictions = np.array(predictions)
        targets = np.array(targets)

        mse = mean_squared_error(targets, predictions)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(targets, predictions)

        self.results['cnn'] = {
            'rmse': rmse,
            'mae': mae,
            'predictions': predictions.tolist(),
            'targets': targets.tolist(),
            'num_samples': len(predictions)
        }

        print(f"CNN Results:")
        print(f"  RMSE: {rmse:.2f} W/m²")
        print(f"  MAE: {mae:.2f} W/m²")
        print(f"  Samples: {len(predictions)}")

        return rmse, mae, predictions, targets

    def evaluate_lstm(self, model_path, test_dataset):
        print("\n=== Evaluating LSTM Forecasting Model ===")
        checkpoint = torch.load(model_path, map_location=self.device, weights_only=False)
        config = checkpoint.get('config', {})
        model = SolarLSTMForecasting(
            input_size=1,
            hidden_size=config.get('lstm_hidden_size', 128),
            num_layers=config.get('lstm_num_layers', 2),
            output_size=config.get('forecast_horizon', 4),
            dropout=config.get('dropout', 0.2)
        ).to(self.device)

        safe_load_checkpoint(model, model_path, self.device)
        model.eval()
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)
        all_predictions, all_targets = [], []
        with torch.no_grad():
            for sequences, targets in test_loader:
                sequences, targets = sequences.to(self.device), targets.to(self.device)
                outputs = model(sequences)
                all_predictions.append(outputs.cpu().numpy())
                all_targets.append(targets.cpu().numpy())

        predictions = np.concatenate(all_predictions, axis=0)
        targets = np.concatenate(all_targets, axis=0)

        overall_rmse = np.sqrt(mean_squared_error(targets.flatten(), predictions.flatten()))
        overall_mae = mean_absolute_error(targets.flatten(), predictions.flatten())

        forecast_step_metrics = []
        if predictions.shape[1] > 0:
            forecast_horizon = predictions.shape[1]
            for step in range(forecast_horizon):
                step_rmse = np.sqrt(mean_squared_error(targets[:, step], predictions[:, step]))
                step_mae = mean_absolute_error(targets[:, step], predictions[:, step])
                forecast_step_metrics.append({
                    'step': step + 1,
                    'rmse': step_rmse,
                    'mae': step_mae
                })

        print("\n=== Overall Metrics ===")
        print(f"RMSE: {overall_rmse:.4f}")
        print(f"MAE : {overall_mae:.4f}")

        self.results['lstm'] = {
            'overall_rmse': overall_rmse,
            'overall_mae': overall_mae,
            'predictions': predictions.tolist(),
            'targets': targets.tolist(),
            'num_samples': len(predictions),
            'step_metrics': forecast_step_metrics # Add step metrics
        }

        return overall_rmse, overall_mae, predictions, targets

    def evaluate_hybrid(self, model_path, test_dataset):
        print("\n=== Evaluating Hybrid CNN-LSTM Model ===")
        checkpoint = torch.load(model_path, map_location=self.device, weights_only=False)
        config = checkpoint.get('config', {})
        model, _ = self.load_model(
            HybridCNNLSTM,
            model_path,
            sequence_length=config.get('sequence_length', 20),
            lstm_hidden_size=config.get('lstm_hidden_size', 128),
            forecast_horizon=config.get('forecast_horizon', 4)
        )

        test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=2)
        all_nowcast_pred, all_nowcast_target, all_forecast_pred, all_forecast_target = [], [], [], []
        with torch.no_grad():
            for image_sequences, historical_irradiance, future_irradiance in test_loader:
                image_sequences = image_sequences.to(self.device)
                historical_irradiance = historical_irradiance.to(self.device)
                future_irradiance = future_irradiance.to(self.device)
                nowcasts, forecasts = model(image_sequences)
                all_nowcast_pred.append(nowcasts.squeeze(-1).cpu().numpy())
                all_nowcast_target.append(historical_irradiance.squeeze(-1).cpu().numpy())
                all_forecast_pred.append(forecasts.cpu().numpy())
                all_forecast_target.append(future_irradiance.cpu().numpy())

        nowcast_pred = np.concatenate(all_nowcast_pred, axis=0)
        nowcast_target = np.concatenate(all_nowcast_target, axis=0)
        forecast_pred = np.concatenate(all_forecast_pred, axis=0)
        forecast_target = np.concatenate(all_forecast_target, axis=0)

        nowcast_rmse = np.sqrt(mean_squared_error(nowcast_target.flatten(), nowcast_pred.flatten()))
        nowcast_mae = mean_absolute_error(nowcast_target.flatten(), nowcast_pred.flatten())
        forecast_rmse = np.sqrt(mean_squared_error(forecast_target.flatten(), forecast_pred.flatten()))
        forecast_mae = mean_absolute_error(forecast_target.flatten(), forecast_pred.flatten())

        forecast_step_metrics = []
        forecast_horizon = forecast_pred.shape[1]
        for step in range(forecast_horizon):
            step_rmse = np.sqrt(mean_squared_error(forecast_target[:, step], forecast_pred[:, step]))
            step_mae = mean_absolute_error(forecast_target[:, step], forecast_pred[:, step])
            forecast_step_metrics.append({
                'step': step + 1,
                'rmse': step_rmse,
                'mae': step_mae
            })

        self.results['hybrid'] = {
            'nowcast_rmse': nowcast_rmse,
            'nowcast_mae': nowcast_mae,
            'forecast_rmse': forecast_rmse,
            'forecast_mae': forecast_mae,
            'forecast_step_metrics': forecast_step_metrics,
            'num_samples': len(forecast_pred)
        }

        print(f"Hybrid Results:")
        print(f"  Nowcast RMSE: {nowcast_rmse:.2f} W/m²")
        print(f"  Nowcast MAE: {nowcast_mae:.2f} W/m²")
        print(f"  Forecast RMSE: {forecast_rmse:.2f} W/m²")
        print(f"  Forecast MAE: {forecast_mae:.2f} W/m²")
        print(f"  Samples: {len(forecast_pred)}")

        return (nowcast_rmse, nowcast_mae, forecast_rmse, forecast_mae,
                forecast_step_metrics, nowcast_pred, forecast_pred)


    def plot_results(self, save_dir='evaluation_plots'):
        """Create visualization plots"""
        os.makedirs(save_dir, exist_ok=True)

        # CNN scatter plot
        if 'cnn' in self.results:
            plt.figure(figsize=(10, 8))

            targets = np.array(self.results['cnn']['targets'])
            predictions = np.array(self.results['cnn']['predictions'])

            plt.subplot(2, 2, 1)
            plt.scatter(targets, predictions, alpha=0.6, s=20)
            plt.plot([targets.min(), targets.max()], [targets.min(), targets.max()], 'r--', lw=2)
            plt.xlabel('Actual Irradiance (W/m²)')
            plt.ylabel('Predicted Irradiance (W/m²)')
            plt.title(f'CNN Nowcasting (RMSE: {self.results["cnn"]["rmse"]:.2f} W/m²)')
            plt.grid(True, alpha=0.3)

            # Residuals plot
            plt.subplot(2, 2, 2)
            residuals = predictions - targets
            plt.scatter(targets, residuals, alpha=0.6, s=20)
            plt.axhline(y=0, color='r', linestyle='--')
            plt.xlabel('Actual Irradiance (W/m²)')
            plt.ylabel('Residuals (W/m²)')
            plt.title('CNN Residuals')
            plt.grid(True, alpha=0.3)

            plt.tight_layout()
            plt.savefig(f'{save_dir}/cnn_evaluation.png', dpi=300, bbox_inches='tight')
            plt.show()
            plt.close()

        # LSTM overall performance
        if 'lstm' in self.results:
            overall_rmse = self.results['lstm']['overall_rmse']
            overall_mae = self.results['lstm']['overall_mae']

            plt.figure(figsize=(8, 6))
            bar_labels = ['RMSE', 'MAE']
            error_values = [overall_rmse, overall_mae]

            plt.bar(bar_labels, error_values, color=['skyblue', 'lightcoral'], alpha=0.7)
            plt.ylabel('Error (W/m²)')
            plt.title('LSTM Overall Performance')
            plt.grid(True, alpha=0.3, axis='y')

            # Add value labels on bars
            for i, val in enumerate(error_values):
                plt.text(i, val + 1, f'{val:.2f}', ha='center', va='bottom')

            plt.tight_layout()
            plt.savefig(f'{save_dir}/lstm_overall_evaluation.png', dpi=300, bbox_inches='tight')
            plt.show()
            plt.close()


        # Model comparison
        if len(self.results) > 1:
            plt.figure(figsize=(12, 8))

            models = []
            rmse_values = []
            mae_values = []

            if 'cnn' in self.results:
                models.append('CNN\nNowcasting')
                rmse_values.append(self.results['cnn']['rmse'])
                mae_values.append(self.results['cnn']['mae'])

            if 'lstm' in self.results:
                models.append('LSTM\nForecasting')
                rmse_values.append(self.results['lstm']['overall_rmse'])
                mae_values.append(self.results['lstm']['overall_mae'])

            if 'hybrid' in self.results:
                models.append('Hybrid\nNowcast')
                rmse_values.append(self.results['hybrid']['nowcast_rmse'])
                mae_values.append(self.results['hybrid']['nowcast_mae'])

                models.append('Hybrid\nForecast')
                rmse_values.append(self.results['hybrid']['forecast_rmse'])
                mae_values.append(self.results['hybrid']['forecast_mae'])

            x = np.arange(len(models))
            width = 0.35

            plt.subplot(1, 1, 1)
            plt.bar(x - width/2, rmse_values, width, label='RMSE', color='skyblue', alpha=0.8)
            plt.bar(x + width/2, mae_values, width, label='MAE', color='lightcoral', alpha=0.8)

            plt.xlabel('Models')
            plt.ylabel('Error (W/m²)')
            plt.title('Model Performance Comparison')
            plt.xticks(x, models)
            plt.legend()
            plt.grid(True, alpha=0.3)

            # Add value labels on bars
            for i, (rmse, mae) in enumerate(zip(rmse_values, mae_values)):
                plt.text(i - width/2, rmse + 1, f'{rmse:.1f}', ha='center', va='bottom')
                plt.text(i + width/2, mae + 1, f'{mae:.1f}', ha='center', va='bottom')

            plt.tight_layout()
            plt.savefig(f'{save_dir}/model_comparison.png', dpi=300, bbox_inches='tight')
            plt.show()
            plt.close()

        print(f"\nPlots saved to {save_dir}/")

    def save_results(self, filename='evaluation_results.json'):
        results_with_metadata = {
            'evaluation_timestamp': datetime.now().isoformat(),
            'device': str(self.device),
            'results': self.results
        }
        with open(filename, 'w') as f:
            json.dump(results_with_metadata, f, indent=2)
        print(f"\nResults saved to {filename}")

    def generate_report(self, filename='evaluation_report.txt'):
        with open(filename, 'w') as f:
            f.write("="*80 + "\n")
            f.write("SOLAR IRRADIANCE FORECASTING - MODEL EVALUATION REPORT\n")
            f.write("="*80 + "\n")
            f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Device: {self.device}\n\n")
            if 'cnn' in self.results:
                r = self.results['cnn']
                f.write("CNN NOWCASTING MODEL\n")
                f.write("-"*30 + "\n")
                f.write(f"RMSE: {r['rmse']:.2f} W/m²\n")
                f.write(f"MAE: {r['mae']:.2f} W/m²\n\n")
            if 'lstm' in self.results:
                r = self.results['lstm']
                f.write("LSTM FORECASTING MODEL\n")
                f.write("-"*30 + "\n")
                f.write(f"RMSE: {r['overall_rmse']:.2f} W/m²\n")
                f.write(f"MAE: {r['overall_mae']:.2f} W/m²\n\n")
            if 'hybrid' in self.results:
                r = self.results['hybrid']
                f.write("HYBRID CNN-LSTM MODEL\n")
                f.write("-"*30 + "\n")
                f.write(f"Nowcast RMSE: {r['nowcast_rmse']:.2f} W/m²\n")
                f.write(f"Nowcast MAE: {r['nowcast_mae']:.2f} W/m²\n")
                f.write(f"Forecast RMSE: {r['forecast_rmse']:.2f} W/m²\n")
                f.write(f"Forecast MAE: {r['forecast_mae']:.2f} W/m²\n\n")
        print(f"Report saved to {filename}")




In [44]:
evaluator = ModelEvaluator()
model_paths = {
    'cnn': r'/content/drive/MyDrive/models/best_cnn_model.pth',
    'lstm': r'/content/drive/MyDrive/models(LSTM)/best_lstm_model.pth',
    'hybrid': r'/content/best_hybrid_model.pth'
}
test_config = {
    'image_dir': r'/content/drive/MyDrive/GIRASOL_DATASET/2019_01_20/infrared',
    'irradiance_file': r'/content/drive/MyDrive/GIRASOL_DATASET/2019_01_20/pyranometer/2019_01_20.csv',
    'sequence_length': 20,
    'forecast_horizon': 4
}

Using device: cuda


In [ ]:
if os.path.exists(model_paths['cnn']):
  baseline_model, _ = evaluator.load_model(EfficientNetRegression, model_paths['cnn'])
  test_dataset = MultiDayDataset(
        image_paths=[os.path.join(test_config['image_dir'], f) for f in sorted(os.listdir(test_config['image_dir'])) if f.endswith(('.png', '.jpg', '.jpeg'))],
        irradiance_values=pd.read_csv(test_config['irradiance_file']).iloc[:, 1].values.astype(np.float32)
    )
  preds, targets = evaluator.evaluate_model(baseline_model, "Baseline CNN", test_dataset)
  evaluator.plot_results() # Re-adding the plotting call

In [39]:
if os.path.exists(model_paths['cnn']):
    print("Preparing CNN test dataset...")
    cnn_test_dataset = MultiDayDataset(
        image_paths=[os.path.join(test_config['image_dir'], f) for f in sorted(os.listdir(test_config['image_dir'])) if f.endswith(('.png', '.jpg', '.jpeg'))],
        irradiance_values=pd.read_csv(test_config['irradiance_file']).iloc[:, 1].values.astype(np.float32)
    )
    evaluator.evaluate_cnn(model_paths['cnn'], cnn_test_dataset)

Preparing CNN test dataset...

=== Evaluating CNN Nowcasting Model ===
CNN Results:
  RMSE: 44.83 W/m²
  MAE: 35.90 W/m²
  Samples: 1712


In [40]:
if os.path.exists(model_paths['lstm']):
    print("Preparing LSTM test dataset...")
    lstm_test_dataset = MultiDayTimeSeriesDataset(
        irradiance_file=test_config['irradiance_file'],
        sequence_length=test_config['sequence_length'],
        forecast_horizon=test_config['forecast_horizon']
    )
    evaluator.evaluate_lstm(model_paths['lstm'], lstm_test_dataset)

Preparing LSTM test dataset...

=== Evaluating LSTM Forecasting Model ===

=== Overall Metrics ===
RMSE: 27.3672
MAE : 16.3862


In [ ]:
if os.path.exists(model_paths['hybrid']):
    print("Preparing Hybrid test dataset...")
    hybrid_test_dataset = SolarSequenceDataset(
        image_dir=test_config['image_dir'],
        irradiance_file=test_config['irradiance_file'],
        sequence_length=test_config['sequence_length'],
        forecast_horizon=test_config['forecast_horizon']
    )
    evaluator.evaluate_hybrid(model_paths['hybrid'], hybrid_test_dataset)

In [43]:
evaluator.plot_results()
evaluator.save_results()
evaluator.generate_report()

print("\n=== Evaluation Complete ===")
print("Check:")
print("- evaluation_results.json")
print("- evaluation_report.txt")
print("- evaluation_plots/")


Plots saved to evaluation_plots/

Results saved to evaluation_results.json
Report saved to evaluation_report.txt

=== Evaluation Complete ===
Check:
- evaluation_results.json
- evaluation_report.txt
- evaluation_plots/


In [ ]:
import torch
print(torch.__version__)


2.8.0+cu126
